In [1]:
'''
!conda install --channel conda-forge geopandas --yes

A script to extract useful information from topoJSON files. 



Author: D. Ellis 2021
'''
None

In [2]:
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
gdf = gpd.GeoDataFrame.from_file('topography.json') # lad boundaries 2020
gdf.drop('id', axis=1,inplace=True)
gdf.head()


,AREACD,AREANM,geometry
0,E06000060,Buckinghamshire,"POLYGON ((-0.84276 51.54472, -0.84704 51.54522..."
1,E06000001,Hartlepool,"POLYGON ((-1.24097 54.72322, -1.22496 54.71480..."
2,E06000002,Middlesbrough,"MULTIPOLYGON (((-1.19432 54.57436, -1.19059 54..."
3,E06000003,Redcar and Cleveland,"POLYGON ((-0.79431 54.55844, -0.79555 54.55489..."
4,E06000004,Stockton-on-Tees,"POLYGON ((-1.25063 54.62535, -1.24567 54.62601..."


In [4]:
''' Bounding Boxes '''
gdf = gdf.merge(gdf.geometry.bounds,left_index=True,right_index=True)

gdf.head()

,AREACD,AREANM,geometry,minx,miny,maxx,maxy
0,E06000060,Buckinghamshire,"POLYGON ((-0.84276 51.54472, -0.84704 51.54522...",-1.140764,51.485807,-0.476706,52.081553
1,E06000001,Hartlepool,"POLYGON ((-1.24097 54.72322, -1.22496 54.71480...",-1.380934,54.621971,-1.164091,54.727176
2,E06000002,Middlesbrough,"MULTIPOLYGON (((-1.19432 54.57436, -1.19059 54...",-1.285556,54.501089,-1.137313,54.590699
3,E06000003,Redcar and Cleveland,"POLYGON ((-0.79431 54.55844, -0.79555 54.55489...",-1.200944,54.487805,-0.794311,54.644085
4,E06000004,Stockton-on-Tees,"POLYGON ((-1.25063 54.62535, -1.24567 54.62601...",-1.452295,54.464206,-1.158707,54.645240


In [5]:
''' Centroid '''
centroids = gdf.centroid
gdf['lat'] = centroids.y
gdf['lon'] = centroids.x

gdf.head()

,AREACD,AREANM,geometry,minx,miny,maxx,maxy,lat,lon
0,E06000060,Buckinghamshire,"POLYGON ((-0.84276 51.54472, -0.84704 51.54522...",-1.140764,51.485807,-0.476706,52.081553,51.785322,-0.812105
1,E06000001,Hartlepool,"POLYGON ((-1.24097 54.72322, -1.22496 54.71480...",-1.380934,54.621971,-1.164091,54.727176,54.669478,-1.259154
2,E06000002,Middlesbrough,"MULTIPOLYGON (((-1.19432 54.57436, -1.19059 54...",-1.285556,54.501089,-1.137313,54.590699,54.541991,-1.222434
3,E06000003,Redcar and Cleveland,"POLYGON ((-0.79431 54.55844, -0.79555 54.55489...",-1.200944,54.487805,-0.794311,54.644085,54.551701,-1.020910
4,E06000004,Stockton-on-Tees,"POLYGON ((-1.25063 54.62535, -1.24567 54.62601...",-1.452295,54.464206,-1.158707,54.645240,54.561628,-1.332462


In [6]:
'''
Lets find all neigbouring areas and overlaps
'''

'\nLets find all neigbouring areas and overlaps\n'

In [7]:
# expandMultiPoly
df = gdf.explode(index_parts=True)

df.geometry = df.buffer(0)

names = gdf.AREANM.values
output = dict(zip(names,[[]]*len(names)))

for index, row in df.iterrows():  
    
    
    neighbors = np.array(df[df.geometry.touches(row['geometry'])].AREANM)
    overlap = np.array(df[df.geometry.overlaps(row['geometry'])].AREANM)
    neighbors = set(np.union1d(neighbors, overlap))
 
    output[row.AREANM]= list((set(output[row.AREANM]) | neighbors )- set([row.AREANM]))



gdf['neighbors']  = [output[x] for x in gdf.AREANM]
gdf.head()


,AREACD,AREANM,geometry,minx,miny,maxx,maxy,lat,lon,neighbors
0,E06000060,Buckinghamshire,"POLYGON ((-0.84276 51.54472, -0.84704 51.54522...",-1.140764,51.485807,-0.476706,52.081553,51.785322,-0.812105,"[Milton Keynes, Cherwell, South Oxfordshire, T..."
1,E06000001,Hartlepool,"POLYGON ((-1.24097 54.72322, -1.22496 54.71480...",-1.380934,54.621971,-1.164091,54.727176,54.669478,-1.259154,"[Stockton-on-Tees, County Durham]"
2,E06000002,Middlesbrough,"MULTIPOLYGON (((-1.19432 54.57436, -1.19059 54...",-1.285556,54.501089,-1.137313,54.590699,54.541991,-1.222434,"[Stockton-on-Tees, Hambleton, Redcar and Cleve..."
3,E06000003,Redcar and Cleveland,"POLYGON ((-0.79431 54.55844, -0.79555 54.55489...",-1.200944,54.487805,-0.794311,54.644085,54.551701,-1.020910,"[Middlesbrough, Hambleton, Scarborough]"
4,E06000004,Stockton-on-Tees,"POLYGON ((-1.25063 54.62535, -1.24567 54.62601...",-1.452295,54.464206,-1.158707,54.645240,54.561628,-1.332462,"[Hartlepool, Hambleton, County Durham, Darling..."


In [8]:
''' Remove unwanted columns and write to file '''
unwanted = ['geometry']

for i in unwanted:
    try:gdf.drop(i,axis=1,inplace=True)
    except:None
        
    
gdf.to_csv('area_summary.csv',index=False)

In [9]:
'Fi'


'Fi'